# Introduction
Python의 Model I/O에 대해 학습함

# FewShotPromptTemplate

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

C:\Users\82102\AppData\Local\Temp\ipykernel_19752\634784058.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [2]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [6]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt

In [7]:
chain = prompt | chat

chain.invoke({"country": "Spain"})

AI:  
I know this:  
Capital: Madrid  
Language: Spanish  
Food: Paella and Tapas  
Currency: Euro  

AIMessage(content='AI:  \nI know this:  \nCapital: Madrid  \nLanguage: Spanish  \nFood: Paella and Tapas  \nCurrency: Euro  ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--33c1ebbf-8792-4d77-8681-f196d4a8995d-0')

# FewShotChatMessagePromptTemplate


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}"),
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you kjnow about {country}?")
  ]
)

In [11]:
chain= final_prompt | chat

chain.invoke(
  {
    "country" : "Korea"
  }
)


        I know this:
        Divided into two countries: South Korea and North Korea
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Language: Korean
        Currency (South Korea): South Korean Won
        Currency (North Korea): North Korean Won
        

AIMessage(content='\n        I know this:\n        Divided into two countries: South Korea and North Korea\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Language: Korean\n        Currency (South Korea): South Korean Won\n        Currency (North Korea): North Korean Won\n        ', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--5f4bbfab-caef-4e5f-9187-a0d461814177-0')

# LengthBasedExampleSelector

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    
]

In [13]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=80,
)

In [14]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [15]:
chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: Here is what I know about Spain:

- **Capital**: Madrid
- **Language**: Spanish (Castilian)
- **Food**: Tapas, Paella, Jamón Ibérico
- **Currency**: Euro
- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history.
- **Geography**: Located in Southwestern Europe, bordered by France to the northeast and Portugal to the west. It has diverse landscapes, including beaches, mountains, and plains.

AIMessage(content='AI: Here is what I know about Spain:\n\n- **Capital**: Madrid\n- **Language**: Spanish (Castilian)\n- **Food**: Tapas, Paella, Jamón Ibérico\n- **Currency**: Euro\n- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history.\n- **Geography**: Located in Southwestern Europe, bordered by France to the northeast and Portugal to the west. It has diverse landscapes, including beaches, mountains, and plains.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--0cc92c35-cf65-4f95-97f1-fb1b1a347718-0')

# Random Example Selector


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [17]:
class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples
  
  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice

    return [choice(self.examples)]

In [18]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

In [19]:
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ],
)

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [23]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

C:\Users\82102\AppData\Local\Temp\ipykernel_19752\423968787.py:42: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  full_prompt = PipelinePromptTemplate(


Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--13bbc185-7732-4f1e-b635-a6deb50dc1a2-0')

# Caching

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  # streaming=True,
  # callbacks=[
  #   StreamingStdOutCallbackHandler(),
  # ]
)


chat.predict("How do you make italian pasta")

C:\Users\82102\AppData\Local\Temp\ipykernel_19752\1912677993.py:20: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("How do you make italian pasta")


[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [llm:ChatOpenAI] [11.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or \"00\" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well:** On a clean surface or in a large bowl, mound the flour and create a well in the center.\n- **Add Eggs:** Crack the eggs into the well and add a pinch of salt.\n- **Mix:** Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n#### 2. Knead the Dough:\n- **Bring Together:** Use your hands to bring the dough together into a ball

'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or "00" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well:** On a clean surface or in a large bowl, mound the flour and create a well in the center.\n- **Add Eggs:** Crack the eggs into the well and add a pinch of salt.\n- **Mix:** Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n#### 2. Knead the Dough:\n- **Bring Together:** Use your hands to bring the dough together into a ball.\n- **Knead:** Knead the dough for about 8-10 minutes until it is smooth and elastic. If the dough is too dry, add a little water; if it’s too sticky, sprinkle with a bit more flour.\n- **Rest:** Wrap the dough in plastic wrap or cover it wi

In [25]:
chat.predict("How do you make italian pasta")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or \"00\" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well:** On a clean surface or in a large bowl, mound the flour and create a well in the center.\n- **Add Eggs:** Crack the eggs into the well and add a pinch of salt.\n- **Mix:** Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n#### 2. Knead the Dough:\n- **Bring Together:** Use your hands to bring the dough together into a ball.\n- *

'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or "00" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well:** On a clean surface or in a large bowl, mound the flour and create a well in the center.\n- **Add Eggs:** Crack the eggs into the well and add a pinch of salt.\n- **Mix:** Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n#### 2. Knead the Dough:\n- **Bring Together:** Use your hands to bring the dough together into a ball.\n- **Knead:** Knead the dough for about 8-10 minutes until it is smooth and elastic. If the dough is too dry, add a little water; if it’s too sticky, sprinkle with a bit more flour.\n- **Rest:** Wrap the dough in plastic wrap or cover it wi

# Serialization


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(True)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
)

with get_openai_callback() as usage:
  a = chat.predict("What is the recipe for soju")
  b = chat.predict("What is the recipe for bread")

  # will be include usage that avobe on this line
  print(usage)
  print(usage.total_cost) # 전체 비용
  print(usage.total_tokens) # 토큰 총합
  print(usage.prompt_tokens) # 프롬프트에 사용된
  print(usage.completion_tokens) # 모델이 쓴 토큰

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [llm:ChatOpenAI] [9.64s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Soju is a traditional Korean distilled spirit, typically made from grains or starches. The exact recipe can vary, but here’s a basic overview of how soju is traditionally made:\n\n### Ingredients:\n1. **Base Ingredient**: Commonly rice, barley, wheat, or sweet potatoes.\n2. **Water**: Clean, filtered water is essential.\n3. **Yeast**: For fermentation.\n4. **Nuruk**: A traditional Korean fermentation starter made from wheat, barley, or rice that contains natural enzymes and yeast.\n\n### Basic Steps to Make Soju:\n\n1. **Preparation of Ingredients**:\n   - If using grains, wash and soak them in water for several hours. Then, steam or cook them until they are soft.\n\n2. **Fermentation**:\n   - Mix the cooked grains with water and add nuruk. This mixtur

In [ ]:
# Legacy code

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#   temperature=0.1,
#   max_tokens=450,
#   model="gpt-4o-mini"
# )

# chat.save("model.json")

chat = load_llm("model.json")
